In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from typing import Sequence

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [3]:
# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

In [4]:
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        Web search agent: Searches for information
        Data analyst: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="A web search agent.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="A data analyst agent. Useful for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    """,
)

In [5]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    termination_condition=termination,
)

In [6]:
task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- WebSearchAgent ----------
[FunctionCall(id='call_P159pzIh27C0dhGFLBdEsU0K', arguments='{"query": "Miami Heat highest points player 2006-2007 season"}', name='search_web_tool'), FunctionCall(id='call_ZF7kCCq1fe57YSfBnmghywbr', arguments='{"query": "Miami Heat player total rebounds 2007-2008 season stats"}', name='search_web_tool'), FunctionCall(id='call_QRIJUrcg3QFGxN8c89vlTw1c', arguments='{"query": "Miami Heat player total rebounds 2008-2009 season stats"}', name='search_web_tool')]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 1397 points\n        James Posey: 550 points\n        ...\n        ', call_id='call_

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), ToolCallRequestEvent(source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=142, completion_tokens=97), content=[FunctionCall(id='call_P159pzIh27C0dhGFLBdEsU0K', arguments='{"query": "Miami Heat highest points player 2006-2007 season"}', name='search_web_tool'), FunctionCall(id='call_ZF7kCCq1fe57YSfBnmghywbr', arguments='{"query": "Miami Heat player total rebounds 2007-2008 season stats"}', name='search_web_tool'), FunctionCall(id='call_QRIJUrcg3QFGxN8c89vlTw1c', arguments='{"query": "Miami Heat player total rebounds 2008-2009 season stats"}', name='search_web_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='WebSearchAgent', models_usage=None, content=[FunctionExecutionResult(cont

In [7]:
def selector_func(messages: Sequence[AgentEvent | ChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None


# Reset the previous team and run the chat again with the selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    termination_condition=termination,
    selector_func=selector_func,
)

await Console(team.run_stream(task=task))

---------- user ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- PlanningAgent ----------
To accomplish this task, I will break it down into manageable subtasks:

1. Web search agent: Find out which Miami Heat player scored the highest points in the 2006-2007 season.
2. Data analyst: Gather the total rebounds for the identified player in the 2007-2008 and 2008-2009 seasons and calculate the percentage change in total rebounds between these two seasons.

Let's begin with the task assignments. 

1. Web search agent: Find out which Miami Heat player scored the highest points in the 2006-2007 season.
2. Data analyst: Gather the total rebounds for the identified player in the 2007-2008 and 2008-2009 seasons and calculate the percentage change in total rebounds between these two seasons. 

Once these tasks are completed, I will summarize th

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=159, completion_tokens=173), content="To accomplish this task, I will break it down into manageable subtasks:\n\n1. Web search agent: Find out which Miami Heat player scored the highest points in the 2006-2007 season.\n2. Data analyst: Gather the total rebounds for the identified player in the 2007-2008 and 2008-2009 seasons and calculate the percentage change in total rebounds between these two seasons.\n\nLet's begin with the task assignments. \n\n1. Web search agent: Find out which Miami Heat player scored the highest points in the 2006-2007 season.\n2. Data analyst: Gather the total rebounds for the identified player in the 2007-200